Programming is hard.  The key to taming its challenges is to divide a complex program into smaller pieces, where each piece is easier to understand.  Different pieces might be divvied up among the different coders on a team, but careful division of a large code base even makes sense for a lone programmer.  If you don't believe us, try looking back at your code from one of the early labs and see how well you remember it!  Careful program design makes it easy for an informed coder to understand and modify a program.  When the program has well-defined top-level structure, often we can tell that we only need to make changes in one piece.  Then we only need to understand that piece.

We've already seen decomposing a program into functions.  Another essential tool of decomposition is *encapsulation*, associated with the idea of *abstract data types (ADTs)*.  An ADT implementation connects a data structure with a set of useful operations on that structure.  The idea is that the details of the data structure are *hidden* from code using the ADT.  That way, the ADT implementation can be modified *independently* from all the other bits of code that depend on it.  You can imagine that one programmer is responsible for the ADT, and another programmer is responsible for coding some algorithm that uses the ADT.  These two programmers can now go about their business simultaneously, potentially finishing the project twice as fast as if they hadn't chosen this decomposition!  The story is even better when the ADT implementation and algorithm are both evolving over time, and each can be improved without affecting the other.  Again, this advantage is significant even with a lone programmer.

OK, enough generalities.  Let's look at some examples.

# Graph Search, Parameterized on Path Data Structures

You've practiced graph pathfinding in Lab 2 and backtracking search in Lab 4.  As a running example throughout this lecture, we'll use a few naive recursive-backtracking approaches to pathfinding.  We'll work with graphs like this one, represented simply as a dictionary mapping nodes to lists of nodes reachable in one step.

In [1]:
graph = {'A': ['B', 'C'],
         'B': ['C', 'D'],
         'C': ['D'],
         'D': ['C'],
         'E': ['F'],
         'F': ['C']}

Our recursive algorithm will maintain a partial path as it goes, and we can use this class to encapsulate the operations we need on paths.

In [2]:
class Path:
    def __init__(self, path=[]):
        self.path = path

    def add(self, node):
        return Path(self.path + [node])

    def visited(self, node):
        return node in self.path

    def len(self):
        return len(self.path)

    def as_list(self):
        return self.path

Now here's the main algorithm.  Notice that this version is not guaranteed to find the *shortest* path!

In [3]:
def find_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if not path.visited(node):
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

def find_path_main(graph, start, end):
    p = find_path(graph, start, end, Path())
    if p:
        return p.as_list()
    else:
        return p

In [4]:
find_path_main(graph, 'A', 'D')

['A', 'B', 'C', 'D']

Here are two other algorithms, to find *all* paths connecting two nodes and to find the *shortest* path connecting two nodes.  Especially the latter is significantly less efficient than what you probably had to come up with in Lab 2, but it still works well as an example here.

In [ ]:
def find_all_paths(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if not path.visited(node):
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

def find_all_paths_main(graph, start, end):
    return [p.as_list()
            for p in find_all_paths(graph, start, end, Path())]


def find_shortest_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if start not in graph:
        return None
    shortest = None
    for node in graph[start]:
        if not path.visited(node):
            newpath = find_shortest_path(graph, node, end, path)
            if newpath:
                if not shortest or newpath.len() < shortest.len():
                    shortest = newpath
    return shortest

def find_shortest_path_main(graph, start, end):
    p = find_shortest_path(graph, start, end, Path())
    if p:
        return p.as_list()
    else:
        return p

In [ ]:
find_all_paths_main(graph, 'A', 'D')

In [ ]:
find_shortest_path_main(graph, 'A', 'D')

We can engineer instances of pathfinding problems where our current implementation runs much more slowly than it needs to.  For instance, consider a graph like this one.

![a degenerate linear graph](linear.png)

The nodes are consecutive integers, each connected to the number one higher than itself.  Our naive path data structure will run each `visited` check by a complete traversal of the path so far, which takes time proportional to the path length.  With many such checks per search, the time adds up.  Here's some code to demonstrate.

In [ ]:
# To see how inefficient this pathfinding is, let's use this trivial straight-line graph.
def straight_line(n):
    return {i: [i+1] for i in range(n)}

# Python's limit on recursion means we'll need to run several searches to get the times high enough to compare.
def perf_test(n, path):
    x = [find_path(straight_line(n), 0, n, path)
         for i in range(1000)]

In [ ]:
perf_test(1000, Path())
print ('finished')

Note how the performance tester is parameterized on an object that provides path operations.  Why would we bother to do that?  Because next we'll see how easy it is to drop in alternative path implementations!

## Linked-List Paths

Remember that an abstract data type, or ADT, provides a set of *methods* that can be called to interact with *private* data of an object.  Code outside that object isn't supposed to muck with the object's fields directly.

We can specify a path ADT as follows, informally.

__Values:__
- Sequences of nodes

__Operations:__
- Create a new empty path
- Create a new path by adding a node to the end of another one
- Check if a node belongs to a path
- Get the length of a path
- Convert a path to a normal Python list

Note how vague we are about what a "sequence of nodes" really is.  This kind of informal specification is meant to explain to a *client* of a path object how to use that object: what to expect different method calls to return in different situations.  It is important that the client can call the object's methods usefully without knowing what is going on inside!

Let's demonstrate the flexibility of an ADT implementer by creating an alternative implementation based on linked lists.

In [ ]:
class EmptyLinkedPath:
    def add(self, node):
        return NonemptyLinkedPath(node, self)

    def visited(self, node):        
        return False

    def len(self):
        return 0

    def as_list(self):
        return []

class NonemptyLinkedPath:
    def __init__(self, head, tail):
        self.head = head
        self.tail = tail
    
    def add(self, node):
        return NonemptyLinkedPath(node, self)
    
    def visited(self, node):
        return node == self.head or self.tail.visited(node)
    
    def len(self):
        return 1 + self.tail.len()

    def as_list(self):
        return self.tail.as_list() + [self.head]

Now, like magic, we can reuse all three graph algorithms from before, but with our alternative path data type and associated code.  Note that, though we start each call off with an empty path, instances of the nonempty-path class will usually be created internally.

In [ ]:
find_path(graph, 'A', 'D', EmptyLinkedPath()).as_list()

In [ ]:
[p.as_list() for p in find_all_paths(graph, 'A', 'D', EmptyLinkedPath())]

In [ ]:
find_shortest_path(graph, 'A', 'D', EmptyLinkedPath()).as_list()

Have we solved our performance problem on the degenerate linear graph?

In [ ]:
perf_test(500, EmptyLinkedPath())
print('finished')

Apparently not!  This version is even slower, though, interestingly, it can be much more space-efficient for large graphs, thanks to sharing in linked lists.

It's worth emphasizing the key benefit of ADTs that just made an appearance: **code can be written independently of ADT implementation details, allowing us to drop in different ADT implementations later**.

##  A Customized Path Data Structure

We could come up with some alternative path representations that work well for many different shapes of graphs.  Instead, let's try to do even better by designing a path class *customized* to graphs like our linear examples.

To start with, how about paths that start at 0 and progress upward through consecutive integers?

In [ ]:
class LessThanPath:
    def __init__(self, bound=0):
        self.bound = bound
        # ...meaning "all consecutive positive integers less than bound."

    def add(self, node):
        if self.bound == node:
            # Good, the client code followed the pattern we optimize for.
            # The next path member comes right after the last.
            return LessThanPath(node+1)
        else:
            # Doesn't fit the pattern.  Fall back to an unoptimized path.
            p = Path()
            for i in range(0, self.bound):
                p = p.add(i)
            return p.add(node)
            # Notice that here we returned an instance of a different class,
            # which is totally legit!

    def visited(self, node):
        # Nice: this check now runs in time independent of path length!
        return 0 <= node and node < self.bound

    def len(self):
        # Ditto for computing the length.
        return self.bound

    def as_list(self):
        # This method still takes time proportional to the path length.
        # You can't win 'em all!
        return list(range(self.bound))

In [ ]:
perf_test(2000, LessThanPath())
print('finished')

In [ ]:
class RangePath:
    def __init__(self, lower=0, upper=0):
        self.lower = lower
        self.upper = upper
        # ...meaning "all integers in [lower, upper)."
        # For the default values of 0 and 0, that's an empty interval!

    def add(self, node):
        if self.lower == self.upper:
            # Empty range so far.  Switch it now to a single-element range.
            return RangePath(node, node+1)
        elif node == self.upper:
            # The range is being expanded by one position higher.
            return RangePath(self.lower, node+1)
        else:
            # Doesn't fit the pattern.  Fall back to an unoptimized path.
            p = Path()
            for i in range(self.lower, self.upper):
                p = p.add(i)
            return p.add(node)

    def visited(self, node):
        return self.lower <= node and node < self.upper

    def len(self):
        return self.upper - self.lower

    def as_list(self):
        return list(range(self.lower, self.upper))

In [ ]:
perf_test(1000, RangePath())

Let's verify that the new implementation also works with a straight-line graph that doesn't begin at zero.

In [ ]:
def straight_line_from(fr, n):
    return {i: [i+1] for i in range(fr, fr+n)}

def perf_test_from(fr, n, path):
    x = [find_path(straight_line_from(fr, n), 0, n, path)
         for i in range(1000)]

In [ ]:
perf_test_from(100, 2000, RangePath())

We have seen another key lesson about ADTs: there is not just one canonical, “best” implementation of each ADT.  Instead, we often **build custom ADT implementations, based on the context of a particular problem**, reusing generic algorithms to work “for free” on our new ADT implementations.

# Graphs as an Abstract Data Type

We can parameterize our graph pathfinding algorithms further, so that they are generic not just in path implementations but also in graph implementations.  Graphs are naturally viewed as an ADT like so:

__Values:__
- Sets of nodes and edges

__Operations:__
- Create a new empty graph
- Add a node to a graph
- Add an edge to a graph
- Check if a node belongs to a graph
- Compute the neighbors (targets of outgoing edges) of a node in a graph

We can easily recast our pathfinding code to work over arbitrary graph implementations.

In [ ]:
def find_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if not graph.hasNode(start):
        return None
    for node in graph.neighbors(start):
        if not path.visited(node):
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

Here is one of the simplest possible implementations of graphs.  Note that it raises `KeyError` exceptions when nonexistent nodes are mentioned.

In [ ]:
class BasicGraph:
    def __init__(self):
        self.nodes = []
        self.edges = []
    
    def addNode(self, n):
        if n not in self.nodes:
            self.nodes.append(n)
    
    def addEdge(self, n1, n2):
        if n1 not in self.nodes or n2 not in self.nodes:
            raise KeyError
        if (n1, n2) not in self.edges:
            self.edges.append((n1, n2))
    
    def hasNode(self, n):
        return n in self.nodes
    
    def neighbors(self, n):
        if n not in self.nodes:
            raise KeyError
        return sorted([n2
                       for n1, n2 in self.edges
                       if n1 == n])
    
    def makeEmpty(self):
        return BasicGraph()

For testing, it will be handy to have an operation to convert our original dictionary-based graph format into whatever format a graph class uses.

In [ ]:
def create_graph(nodes, graph):
    for node, neighbors in nodes.items():
        graph.addNode(node)
        for neighbor in neighbors:
            graph.addEdge(node, neighbor)
    return graph

In [ ]:
find_path(create_graph(graph, BasicGraph()), 'A', 'D', Path()).as_list()

Note how now, to invoke pathfinding, we choose both a graph implementation and a path implementation.  Any pair of correct implementations should lead to the same answer, though performance may vary drastically based on what we select.

# Summary

*Encapsulation*, or information hiding, is one of the big ideas in effective program design.  One common embodiment of the idea is *abstract data types*, which package together operations on some hidden representation type.  We've highlighted a few key points about the versatility of abstract data types:

- **Code can be written generically in some ADT, so that the code works when any implementation of the ADT is plugged in.**  This way, ADT implementations and their client programs can be coded, understood, and maintained separately.
- **A correct implementation of an ADT appears to behave identically to the simple reference implementation on any sequence of method calls.**  That way, the former really is a drop-in replacement for the latter, and programmers may pretend that they are using the simpler reference implementation when they are actually using a more clever implementation.
- There is not just one “best” implementation of any ADT: **we often want to implement one ADT differently for different problem contexts, appealing to the same generic algorithms that work “for free” given any implementation of the ADT.**  These custom implementations often improve performance by taking advantage of restrictions on the data that appear in the problem at hand.